In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import pandas_datareader.data as web
import datetime

In [14]:
#If we use too much data for example back in 2010, lots of data can be deprecated
# so to get our mu and sigma it is better to use data from the past 2-3 years
start_date = datetime.datetime(2014,1,1)
end_date = datetime.datetime(2017,10,15)

S = 1e6 #investment
c = 0.99 #confidence level
n = 1

## Variance Method

In [4]:
#if we want to calculate VaR for tomorrow
def value_at_risk(position,c,mu,sigma):
    alpha = norm.ppf(1-c) #inverse of CDF (cumulative distribution function)
    var = position*(mu-sigma*alpha)
    return var

In [5]:
#we want to calculate VaR in n days time
#we have to consider that the mean and standard deviation will change
# mu = mu*n and sigma = sigma*sqrt(n)
def value_at_risk_long(S,c,mu,sigma,n):
    alpha = norm.ppf(1-c)
    var = S*(mu*n-sigma*np.sqrt(n)*alpha)
    return var

In [6]:
citi = web.DataReader('C', data_source='yahoo', start=start_date, end=end_date) #Citigroup Inc. (C)

In [7]:
citi.head(3)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-01-02,52.400002,51.810001,52.029999,52.270000,16479700.0,48.380669
2014-01-03,53.470001,52.310001,52.389999,53.400002,26884900.0,49.426586
2014-01-06,54.290001,53.430000,53.619999,53.810001,28503100.0,49.806084


In [8]:
citi['returns'] = citi['Adj Close'].pct_change() #Computes the percentage change from the previous row by default

In [9]:
citi.head(3)

,High,Low,Open,Close,Volume,Adj Close,returns
Date,,,,,,,
2014-01-02,52.400002,51.810001,52.029999,52.270000,16479700.0,48.380669,NaN
2014-01-03,53.470001,52.310001,52.389999,53.400002,26884900.0,49.426586,0.021619
2014-01-06,54.290001,53.430000,53.619999,53.810001,28503100.0,49.806084,0.007678


In [10]:
#we can assume daily returns to be normally distributed: mean and variance (standard deviation):
mu = np.mean(citi['returns'])
sigma = np.std(citi['returns'])

print('Value at Risk is: $%0.2f' % value_at_risk(S,c,mu,sigma))

Value at Risk is: $36382.84


In [16]:
#If we possess $1m in citi shares today, the maximum amout of loss tomorrow is $36k, with 99% confidence level

In [15]:
print('Value at Risk is: $%0.2f' % value_at_risk_long(S,c,mu,sigma,n))

Value at Risk is: $36382.84


## Monte-Carlo Method

In [17]:
iterations = 100000 #Monte-Carlo simulations

In [79]:
class ValueAtRiskMonteCarlo:
    
    def __init__(self,S,c,mu,sigma,n,iterations):
        self.S = S
        self.c = c
        self.mu = mu
        self.sigma = sigma
        self.n = n
        self.iterations = iterations
        
    def simulation(self):
        
        stock_data = np.zeros([self.iterations,1])
        rand = np.random.normal(0,1,[1,self.iterations])
        
        #equation for the S(t) stock price
        stock_price = self.S*np.exp(self.n*(self.mu-(self.sigma**2)/2)+self.sigma*np.sqrt(self.n)*rand)
        
        #we have to sort the stock prices to determine the percentile
        stock_price = np.sort(stock_price) #facultatif je pense car la fonction percentile trie deja dans l'ordre

        #it depends on the confidence level: 95% -> 5 and 99% -> 1
        percentile = np.percentile(stock_price,(1-self.c)*100) #Returns the qth percentile(s) of the array elements.
        print('Initial investment:', self.S)
        print('Percentile value: $%0.2f' %percentile)
        return self.S-percentile
    
    #En gros on genere pleins de prix qui correspondent a la valeur que notre investissement S pourrait etre demain
    #avec notre equation. On prend le 1th ou 5th percentile qui correspond a la valeur au dessous de laquelle on a 
    #1% ou 5% des valeurs les plus faibles (donc les simulations pour lesquelles la valeur de notre investissement
    #aurait perdu le plus) et on fini par soustraire ce percentile a notre valeur d'investissement initial S.
    

In [85]:
model = ValueAtRiskMonteCarlo(S,c,mu,sigma,n,iterations)

In [86]:
print('Value at risk with Monte-Carlo simulation: $%0.2f' % model.simulation())

Initial investment: 1000000.0
Percentile value: $964909.76
Value at risk with Monte-Carlo simulation: $35090.24
